This function computes the gradient of the velocity-field $ \mathbf{v}(\mathbf{x}, t) $ using an auxiliary meshgrid.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (Ny, Nx) | X-meshgrid of velocity data|
| Y | array (Ny, Nx) | Y-meshgrid of velocity data|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Ny, Nx) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| time_data | array(1,Nt) | time of velocity data |
| delta_data | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| grad_vel | array (2,2) | $ \nabla \mathbf{v}(\mathbf{x}, t) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add integration folder to current working path
sys.path.append(parent_directory+"/integration")

In [2]:
# Import numpy as np
import numpy as np

# Import package which computes velocity in the domain
from ipynb.fs.defs.velocity import velocity

# Import package which checks particle location
from ipynb.fs.defs.check_location import check_location

In [3]:
def gradient_velocity(t, x, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, dt_data, delta_data, aux_grid):
    
    x = x.reshape(2,-1)
    
    # define auxiliary grid spacing
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    X0, XL, XR, XU, XD = [], [], [], [], []

    nan_mask = []
    
    for i in range(x.shape[1]):
        
        xr = x[0, i] + rho_x
        xl = x[0, i] - rho_x
        
        if periodic[0]:
            xr = xr%(X[0,-1]-X[0,0])+X[0,0]
            xl = xl%(X[0,-1]-X[0,0])+X[0,0]
        
        yu = x[1, i] + rho_y
        yd = x[1, i] - rho_y
        
        if periodic[1]:
            yu = yd%(Y[-1,0]-Y[0,0])+Y[0,0]
            yd = yu%(Y[-1,0]-Y[0,0])+Y[0,0]
        
        bool_xr = (check_location(X, Y, defined_domain, np.array([xr, x[1, i]]))[0] == "IN")
        bool_xl = (check_location(X, Y, defined_domain, np.array([xl, x[1, i]]))[0] == "IN")
        bool_yu = (check_location(X, Y, defined_domain, np.array([x[0, i], yu]))[0] == "IN")
        bool_yd = (check_location(X, Y, defined_domain, np.array([x[0, i], yd]))[0] == "IN")
        
        # check initial location of particles. Only compute gradient of flow map for those particles whose auxiliary trajectories are all within the defined flow domain
        if bool_xr and bool_xl and bool_yu and bool_yd:
            
            nan_mask.append(True)
            
        else:
            
            nan_mask.append(False)
    
        X0.append([x[0, i], x[1, i]])
        XL.append([xl, x[1, i]])
        XR.append([xr, x[1, i]])
        XU.append([x[0, i], yu])
        XD.append([x[0, i], yd])
    
    X0 = np.array(X0).transpose()
    XL = np.array(XL).transpose()
    XR = np.array(XR).transpose()
    XU = np.array(XU).transpose()
    XD = np.array(XD).transpose()
    
    # velocity on the auxiliary meshgrid
    vLend = velocity(t, XL, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, dt_data)    
    vRend = velocity(t, XR, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, dt_data)     
    vDend = velocity(t, XD, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, dt_data)   
    vUend = velocity(t, XU, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, dt_data)
    
    grad_vel = np.zeros((2, 2, vLend.shape[1]))
    
    # compute gradient of flow map over time interval
    
    for i in range(X0.shape[1]):
        
        if nan_mask[i]:

            grad_vel[0, 0, i] = (vRend[0,i]-vLend[0,i])/(2*rho_x)
            grad_vel[1, 0, i] = (vRend[1,i]-vLend[1,i])/(2*rho_x)
            grad_vel[0, 1, i] = (vUend[0,i]-vDend[0,i])/(2*rho_y)
            grad_vel[1, 1, i] = (vUend[1,i]-vDend[1,i])/(2*rho_y)
                
        else:
            
            grad_vel[:,:,i] = np.nan
    
    return grad_vel